In [1]:
import json
import ROOT
from ROOT import RDataFrame, TCanvas, THStack
ROOT.EnableImplicitMT()
import numpy as np
%jsroot on

Welcome to JupyROOT 6.24/02


In [2]:
@ROOT.Numba.Declare(["unsigned int", "ROOT::VecOps::RVec<float>&"], "ROOT::VecOps::RVec<double>")
def btag_weight_variation(i_jet, jet_pt):
    return 1 + np.array((.1,-.1))*(jet_pt[i_jet]/50)

In [3]:
ROOT.gSystem.SetIncludePath("-I$ROOTSYS/include")
ROOT.gSystem.CompileMacro("helper.cpp", "kO")

1

In [4]:
class TtbarAnalysis:

    def __init__(self, n_files_max_per_sample = 1, num_bins=25, bin_low = 50, bin_high = 550):
        
        self.j4b2 = {}
        self.j4b1 = {}
        self.n_files_max_per_sample = n_files_max_per_sample
        with open ('ntuples.json') as f:
            file_info = json.load(f)
        self.fileset = self._construct_fileset()
        
        self.num_bins = num_bins
        self.bin_low = bin_low
        self.bin_high = bin_high
        self.xsec_info = {
            "ttbar": 396.87 + 332.97, # nonallhad + allhad, keep same x-sec for all
            "single_top_s_chan": 2.0268 + 1.2676,
            "single_top_t_chan": (36.993 + 22.175)/0.252,  # scale from lepton filter to inclusive
            "single_top_tW": 37.936 + 37.906,
            "wjets": 61457 * 0.252,  # e/mu+nu final states
            "data": None
        }

        
    def _construct_fileset(self):
        n_files_max_per_sample = self.n_files_max_per_sample
        with open ('ntuples.json') as f:
            file_info = json.load(f)
        fileset = {}
        for process in file_info.keys():
            if process == "data":
                continue  # skip data
            fileset[process] = {}
            for variation in file_info[process].keys():
                if variation != 'nominal': continue
                
                file_list = file_info[process][variation]["files"]
                if n_files_max_per_sample != -1:
                    file_list = file_list[:n_files_max_per_sample]  # use partial set of samples

                file_paths = [f["path"] for f in file_list]
                fileset[process].update({variation: file_paths})
        return fileset
    
    def _j4b1 (self, process, variation):
    
        self.j4b1[process] = {}
        
        d = RDataFrame('events', self.fileset[process][variation])

        x_sec = self.xsec_info[process]
        nevts_total = d.Count().GetValue()
        lumi = 3378 # /pb
        xsec_weight = x_sec * lumi / nevts_total


        d = d.Define('electron_pt_mask', 'electron_pt>25').Define('muon_pt_mask', 'muon_pt>25').Define('jet_pt_mask', 'jet_pt>25')\
             .Filter('Sum(electron_pt_mask) + Sum(muon_pt_mask) == 1')\
             .Filter('Sum(jet_pt_mask) >= 4')\
             .Filter('Sum(jet_btag[jet_pt_mask]>=0.5)==1')            


        d = d.Define('HT', 'Sum(jet_pt[jet_pt_mask])')
        
              
        d = d.Define('weights', str(xsec_weight))
        pt = d.Histo1D(
                       ('pt_'+process+'_'+variation, process, self.num_bins, self.bin_low, self.bin_high), 
                        'HT', 
                        'weights'
                      )
        
        print('histogram for region = j4b1, process = ' + process  + ', variation = ' + variation + ' has been created')
        
        self.j4b1[process][variation] = pt
        
        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
            for i_dir, direction in enumerate(["up", "down"]):
#                 sign = ['+','-']
                variation=f"{weight_name}_{direction}"
                d = d.Define('weights_'+variation, 
#                          f"return (1 {sign[i_dir]} jet_pt[jet_pt_mask][{i_var}]/50/10)*weights;"
                           f"weights*Numba::btag_weight_variation({i_var}, jet_pt[jet_pt_mask])[{i_dir}]"
                        )
                self.j4b1[process][variation] = d.Histo1D(
                                    ('h_'+process+variation, variation, self.num_bins, self.bin_low, self.bin_high),
                                    'HT',
                                    'weights_'+variation
                )
                
        
    def _j4b2 (self, process, variation):
    
        self.j4b2[process] = {}
        
        d = RDataFrame('events', self.fileset[process][variation])

        x_sec = self.xsec_info[process]
        nevts_total = d.Count().GetValue()
        lumi = 3378 # /pb
        xsec_weight = x_sec * lumi / nevts_total


        d = d.Define('electron_pt_mask', 'electron_pt>25').Define('muon_pt_mask', 'muon_pt>25').Define('jet_pt_mask', 'jet_pt>25')\
             .Filter('Sum(electron_pt_mask) + Sum(muon_pt_mask) == 1')\
             .Filter('Sum(jet_pt_mask) >= 4')\
             .Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>=2')            


        jet_p4 = d.Define("jet_p4", 
            "ROOT::VecOps::Construct<ROOT::Math::PxPyPzMVector>(jet_px[jet_pt_mask], jet_py[jet_pt_mask], jet_pz[jet_pt_mask], jet_mass[jet_pt_mask])"
        )

        trijet = jet_p4.Define('trijet', 
            'ROOT::VecOps::Combinations(jet_pt[jet_pt_mask],3)'
        ).Define('ntrijet', 'trijet[0].size()')

        trijet_p4 = trijet.Define('trijet_p4', 
                              'ROOT::VecOps::RVec<ROOT::Math::PxPyPzMVector> trijet_p4(ntrijet);'              +\
                              'for (int i = 0; i < ntrijet; ++i) {'                                            +\
                                  'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'       +\
                                  'trijet_p4[i] = jet_p4[j1] + jet_p4[j2] + jet_p4[j3];'                       +\
                              '}'                                                                              +\
                              'return trijet_p4;'                                                                                                                          
                             )

        #TODO  implement references
        trijet_pt = trijet_p4.Define('trijet_pt', 
                'return ROOT::VecOps::Map(trijet_p4, [](ROOT::Math::PxPyPzMVector v) { return v.Pt(); })'
                                    )

        trijet_pt_btag = trijet_pt.Define('trijet_btag', 
                                          'ROOT::VecOps::RVec<bool> btag(ntrijet);'                                   +\
                                          'for (int i = 0; i < ntrijet; ++i) {'                                       +\
                                           'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'     +\
                                           'btag[i]=std::max({jet_btag[j1], jet_btag[j2], jet_btag[j3]})>0.5;'        +\
                                          '}'                                                                         +\
                                          'return btag;'
                                    )

        trijet_mass=trijet_pt_btag.Define('trijet_mass',
                                          'double mass;'+\
                                          'double Pt = 0;'+\
                                          'double indx = 0;'+\
                                          'for (int i = 0; i < ntrijet; ++i) {'               +\
                                          '    if ((Pt < trijet_pt[i]) && (trijet_btag[i])) {'+\
                                          '        Pt = trijet_pt[i];'+\
                                          '        indx=i;'+\
                                          '    }'                                            +\
                                          '}'                                                +\
                                          'mass = trijet_p4[indx].M();'             +\
                                          'return mass;'
                                         )


        trijet_mass = trijet_mass.Define('weights', str(xsec_weight))
        mass = trijet_mass.Histo1D(('h_'+process+'_'+variation, process, self.num_bins, self.bin_low, self.bin_high), 'trijet_mass', 'weights')
        
        print('histogram for region = j4b2, process = ' + process  + ', variation = ' + variation + ' has been created')
        
        self.j4b2[process][variation] = mass  
        
        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
            for i_dir, direction in enumerate(["up", "down"]):

                variation=f"{weight_name}_{direction}"
                trijet_mass = trijet_mass.Define('weights_'+variation, 
                           f"weights*Numba::btag_weight_variation({i_var}, jet_pt[jet_pt_mask])[{i_dir}]"
                        )
                self.j4b2[process][variation] = trijet_mass.Histo1D(
                                    ('h_'+process+variation, variation, self.num_bins, self.bin_low, self.bin_high),
                                    'trijet_mass',
                                    'weights_'+variation
                )
    
    def fill_j4b2(self):
        for process in self.fileset.keys():
            for variation in self.fileset[process].keys():
                self._j4b2(process=process, variation=variation)
    def fill_j4b1(self):
        for process in self.fileset.keys():
            for variation in self.fileset[process].keys():
                self._j4b1(process=process, variation=variation)   
    def ProcStack_j4b1(self, variation='nominal', rebin = 0):
        hs = THStack('hsb1', '>= 4 jets, 1 b-tag; H_{T} [GeV]')
        for process in self.fileset.keys():
            h = self.j4b1[process][variation]
            if not rebin:
                hs.Add(h.GetPtr())
            else:
                hs.Add(h.Rebin(rebin, h.GetTitle()))
        return hs
    def ProcStack_j4b2(self, variation='nominal', rebin = 0):
        hs = THStack('j4b2', '>=4 jets, >=2 b-tag (RDF); m_{bjj} [GeV]')
        for process in self.fileset.keys():
            h = self.j4b2[process][variation]
            if not rebin:
                hs.Add(h.GetPtr())
            else:
                hs.Add(h.Rebin(rebin, h.GetTitle()))
        return hs
    
    def btagup_j4b1(self, process='ttbar'):       
        up = THStack('j4b1up', 'b-tagging variations; HT [GeV]')
        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
            direction = "up"
            variation=f"{weight_name}_{direction}"
            up.Add(self.j4b1[process][variation].GetPtr())
  
        return up
                     
    def btagdown_j4b1(self, process='ttbar'):

        down = THStack('j4b1down', 'b-tagging variations; HT [GeV]')
        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
            direction = "down"
            variation=f"{weight_name}_{direction}"
            down.Add(self.j4b1[process][variation].GetPtr())        
        return down

    def btag_values(self, process='ttbar'):
        return np.array([get_values(self.j4b1[process][f"btag_var_{i}_{direction}"], 25) for direction in ["up", "down"] for i in range(4)])                   
                     
    def btagup_j4b2(self, process='ttbar'):       
        up = THStack('j4b2up', 'b-tagging variations; HT [GeV]')
        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
            direction = "up"
            variation=f"{weight_name}_{direction}"
            up.Add(self.j4b2[process][variation].GetPtr())
  
        return up
                     
    def btagdown_j4b2(self, process='ttbar'):

        down = THStack('j4b2down', 'b-tagging variations; HT [GeV]')
        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
            direction = "down"
            variation=f"{weight_name}_{direction}"
            down.Add(self.j4b2[process][variation].GetPtr())        
        return down


    
    def list_j4b1(self, variation='nominal'):
        hs = []
        for process in self.fileset.keys():
            h = self.j4b1[process][variation]
            hs.append(h)
        return hs
    def list_j4b2(self, variation='nominal'):
        hs = []
        for process in self.fileset.keys():
            h = self.j4b2[process][variation]
            hs.append(h)
        return hs
    
    def values_j4b1(self, variation='nominal'):
        return np.array([get_values(self.j4b1[process][variation], 25) for process in self.fileset.keys()])
    def values_j4b2(self, variation='nominal'):
        return np.array([get_values(self.j4b2[process][variation], 25) for process in self.fileset.keys()])

In [6]:
def compare(number, rdf_values, coffea_values, precision):
    
    rdf_values = rdf_values[number]
    coffea_values = coffea_values[number]
    rdf_values = np.round(rdf_values, precision)
    coffea_values = np.round(coffea_values, precision)
    mask = rdf_values == coffea_values

    return not (False in mask)

In [7]:
def test_nominal():
    alex4jb1 = np.load('4j1b.npy')
    alex4jb2 = np.load('4j2b.npy')
    rdf_4jb1 = analysisManager.values_j4b1()
    rdf_4jb2 = analysisManager.values_j4b2()
    for i in range(5):
        print(compare(i, rdf_4jb1, alex4jb1, precision=5))
    for i in range(5):
        print(compare(i, rdf_4jb2, alex4jb2, precision=4))

In [8]:
def test_btag(precision=4):
    alex4j1b = np.load('b1tag.npy')
    r4j1b = analysisManager.btag_values()
    r4j1b_swapped = np.concatenate((r4j1b[4:], r4j1b[:4]))
    for i in range(8):
        print(compare(i, r4j1b_swapped, alex4j1b, precision))

In [5]:
analysisManager = TtbarAnalysis()

In [6]:
analysisManager.fileset.keys()

dict_keys(['ttbar', 'single_top_s_chan', 'single_top_t_chan', 'single_top_tW', 'wjets'])

In [7]:
analysisManager.fill_j4b2()

histogram for region = j4b2, process = ttbar, variation = nominal has been created
histogram for region = j4b2, process = single_top_s_chan, variation = nominal has been created
histogram for region = j4b2, process = single_top_t_chan, variation = nominal has been created
histogram for region = j4b2, process = single_top_tW, variation = nominal has been created
histogram for region = j4b2, process = wjets, variation = nominal has been created


In [8]:
analysisManager.fill_j4b1()

histogram for region = j4b1, process = ttbar, variation = nominal has been created
histogram for region = j4b1, process = single_top_s_chan, variation = nominal has been created
histogram for region = j4b1, process = single_top_t_chan, variation = nominal has been created
histogram for region = j4b1, process = single_top_tW, variation = nominal has been created
histogram for region = j4b1, process = wjets, variation = nominal has been created


In [13]:
# test_nominal()

In [14]:
# test_btag(precision=3)

In [15]:
# ROOT.gStyle.SetPalette(ROOT.kSolar)
# c = TCanvas('c', 'title', 1000, 400)
# c.Divide(2,1)
# pad1 = c.cd(1)
# hs2 = analysisManager.ProcStack_j4b2()
# hs2.Draw('pfc hist stack')
# pad1.BuildLegend()
# pad2 = c.cd(2)
# hs1 = analysisManager.ProcStack_j4b1()
# hs1.Draw('pfc hist stack')
# pad2.BuildLegend()
# c.Draw()

In [16]:
# hs = analysisManager.ProcStack_j4b1(rebin=2)
# c = TCanvas()
# hs.Draw('hist pfc plc')
# c.Draw()
# c.BuildLegend(0.65, 0.7, 0.9, 0.9)
# x = hs.GetXaxis()
# x.SetRangeUser(120, 550)
# x.SetTitleOffset(1.5)
# x.CenterTitle()

In [17]:
# hs = analysisManager.ProcStack_j4b2()
# c = TCanvas()
# hs.Draw('hist pfc plc')
# c.Draw()
# c.BuildLegend(0.65, 0.7, 0.9, 0.9)
# x = hs.GetXaxis()
# x.SetTitleOffset(1.5)
# x.CenterTitle()

In [18]:
c = TCanvas('c', 'c', 800, 600) 
hlist = analysisManager.list_j4b1()
hs = THStack('j4b1', '>=4 jets, 1 b-tag (RDF); H_{T} [GeV]')
for h in hlist:
    ptr = h.Rebin(2, h.GetTitle())
    hs.Add(ptr)
hs.Draw('hist pfc plc')
c.Draw()
x = hs.GetXaxis()
x.SetRangeUser(120, 500)
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)
c.SaveAs('graph/reg4j1b.png')

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Info in <TCanvas::Print>: png file graph/reg4j1b.png has been created


In [19]:
hlist = analysisManager.list_j4b2()
hs = THStack('j4b1', '>=4 jets, >=2 b-tag (RDF); m_{bjj} [GeV]')
for h in hlist[1:]:
    hs.Add(h.GetPtr())
hs.Add(hlist[0].GetPtr())
hs.Draw('hist pfc plc')
c.Draw()
x = hs.GetXaxis()
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)
c.SaveAs('graph/reg4j2b.png')

Info in <TCanvas::Print>: png file graph/reg4j2b.png has been created


In [20]:
freshstack = analysisManager.btagup_j4b1()
hs = THStack('j4b1btag', 'b-tagging variations (RDF); H_{T} [GeV]')
for h in freshstack.GetHists():
    ptr = h.Rebin(2, h.GetTitle())
    ptr.SetFillColor(0)
    ptr.SetLineWidth(1)
    hs.Add(ptr)
hs.Draw('hist nostack')
c.Draw()
x = hs.GetXaxis()
x.SetRangeUser(120, 500)
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)
c.SaveAs('graph/btagvar.png')

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Info in <TCanvas::Print>: png file graph/btagvar.png has been created


In [21]:
# print('reg=b1')
# for proc in analysisManager.j4b1.keys():
#     print(proc, end=': ')
#     for var in analysisManager.j4b1[proc].keys():
#         print(var, end = ' ')
#     print()
# print('reg=b2')
# for proc in analysisManager.j4b2.keys():
#     print(proc, end=': ')
#     for var in analysisManager.j4b2[proc].keys():
#         print(var, end = ' ')
#     print()    

In [9]:
output = ROOT.TFile.Open('rdf.root', 'RECREATE')
regions = ["4j1b", "4j2b"]
histograms = [analysisManager.j4b1, analysisManager.j4b2]
for i in range(2):
    region = regions[i]; histogram = histograms[i]
    for process in histogram.keys():
        for variation in histogram[process].keys():
            hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
            hist = histogram[process][variation]
            hist_ptr = hist.Rebin(2, hist.GetTitle())
            hist_ptr = ROOT.Slice(hist_ptr, 120, 550)
            output.WriteObject(hist_ptr, hist_name)
output.Close()

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nb

In [10]:
rdf = ROOT.TFile.Open('rdf.root')
coffea = ROOT.TFile.Open('histograms.root')

In [11]:
rdf.ls()

TFile**		rdf.root	
 TFile*		rdf.root	
  KEY: TH1D	4j1b_ttbar;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_0_up;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_0_down;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_1_up;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_1_down;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_2_up;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_2_down;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_3_up;1	object title
  KEY: TH1D	4j1b_ttbar_btag_var_3_down;1	object title
  KEY: TH1D	4j1b_single_top_s_chan;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_0_up;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_0_down;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_1_up;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_1_down;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_2_up;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_2_down;1	object title
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_3_up;1	object ti

In [12]:
import traceback

def get_values(h):
    return np.array([h.GetBinContent(i+1) for i in range (h.GetNbinsX())])

def match_histos (rdf_h, coffea_h, precision):
    rdf_values = get_values(rdf_h)
    coffea_values = get_values(coffea_h)
    rdf_values = np.round(rdf_values, precision)
    coffea_values = np.round(coffea_values, precision)
    mask = rdf_values == coffea_values

    return not (False in mask)

# def match_histos (rdf_h, coffea_h):
#     rdf_values = get_values(rdf_h)
#     coffea_values = get_values(coffea_h)
#     mask = rdf_values == coffea_values

#     return not (False in mask)

regions = ["4j1b", "4j2b"]
histograms = [analysisManager.j4b1, analysisManager.j4b2]

for i in range(2):
    region = regions[i]; histogram = histograms[i]
    for process in histogram.keys():
        for variation in histogram[process].keys():
            hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
            rdf_hist = rdf.Get(hist_name)
            coffea_hist = coffea.Get(hist_name)
#             print(hist_name,'is processing')
            if ((rdf_hist and coffea_hist) and coffea):
                if not match_histos(rdf_hist, coffea_hist, precision=0):
                    print('mismatch', hist_name)
            else:
                raise ValueError('rdf_hist and coffea_hist is Zombie')

mismatch 4j1b_ttbar
mismatch 4j1b_ttbar_btag_var_0_up
mismatch 4j1b_ttbar_btag_var_0_down
mismatch 4j1b_ttbar_btag_var_1_up
mismatch 4j1b_ttbar_btag_var_1_down
mismatch 4j1b_ttbar_btag_var_2_up
mismatch 4j1b_ttbar_btag_var_2_down
mismatch 4j1b_ttbar_btag_var_3_up
mismatch 4j1b_ttbar_btag_var_3_down
mismatch 4j1b_single_top_s_chan
mismatch 4j1b_single_top_s_chan_btag_var_0_up
mismatch 4j1b_single_top_s_chan_btag_var_0_down
mismatch 4j1b_single_top_s_chan_btag_var_1_up
mismatch 4j1b_single_top_s_chan_btag_var_1_down
mismatch 4j1b_single_top_s_chan_btag_var_2_up
mismatch 4j1b_single_top_s_chan_btag_var_2_down
mismatch 4j1b_single_top_s_chan_btag_var_3_up
mismatch 4j1b_single_top_s_chan_btag_var_3_down
mismatch 4j1b_single_top_t_chan
mismatch 4j1b_single_top_t_chan_btag_var_0_up
mismatch 4j1b_single_top_t_chan_btag_var_0_down
mismatch 4j1b_single_top_t_chan_btag_var_1_up
mismatch 4j1b_single_top_t_chan_btag_var_1_down
mismatch 4j1b_single_top_t_chan_btag_var_2_up
mismatch 4j1b_single_top_t

In [13]:
rdf_ttbar_1 = rdf.Get('4j1b_ttbar')
coffea_ttbar_1 = coffea.Get('4j1b_ttbar')

In [14]:
c = TCanvas('cc', 'cc')

In [15]:
%jsroot on
c.Divide(2,1)
c.cd(1)
rdf_ttbar_1.Draw('hist pfc')
c.cd(2)
coffea_ttbar_1.Draw('hist pfc')
c.Draw()

In [25]:
tt1 = analysisManager.j4b1['ttbar']['nominal']

In [19]:
tt1

<cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x125ec670>

In [20]:
tt1 = tt1.Rebin(2)

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.


In [21]:
tt1

In [26]:
dc = TCanvas('dc', 'dc')
dc.Divide(2,1)
dc.cd(1)
tt1.Draw('hist pfc')
dc.cd(2)
tt1.Rebin(2).Draw('hist pfc')
dc.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: dc


In [27]:
analysisManager._j4b1(process='ttbar', variation='nominal')

histogram for region = j4b1, process = ttbar, variation = nominal has been created
